In [16]:
special_token_id = [0,1,2,3] 
mlm_probability=0.15
import torch
import random

def get_special_tokens_mask(value):
        print("value")
        print(value)
        condition = torch.isin(value, torch.tensor(special_token_id,dtype=torch.int))
        output_tensor = torch.where(condition, torch.zeros_like(value), torch.ones_like(value))
        print("output_tensor")
        print(output_tensor)
        return output_tensor
    
def mask_edge(labels,y,index):
        prob_matrix = []
        for i in range(y.shape[0] - 1):
            edge_number = ((index[0] >= y[i]) & (index[0] < y[i+1])).sum()
            #求出每一个图的边数
            rand = random.randint(0,edge_number-1)
            indices_replaced = torch.bernoulli(torch.full((edge_number, 1), mlm_probability)).bool()
            # probability_matrix[torch.nonzero(indices_replaced == False)[0][0],:]  = 1
            probability_matrix = torch.zeros((edge_number,labels.shape[1]))
            if torch.any(indices_replaced == True):
                probability_matrix[torch.nonzero(indices_replaced == True)[0][0],:]  = 1
            else:
                probability_matrix[rand,:] = 1
            prob_matrix.append(probability_matrix)
        prob_matrix = torch.cat(prob_matrix, dim = 0)
        return prob_matrix

def mask_node(labels, y, special_tokens_mask):
        prob_matrix = []
        print(labels.shape)
        print(y.shape)
        for i in range(y.shape[0] - 1):
            rand = random.randint(0,y[i+1]-y[i]-1)
            # mlm_probability 掩码 node 属性
            indices_replaced = torch.bernoulli( torch.full((y[i + 1] - y[i], 1), mlm_probability) ).bool()
            print("indices_replaced")
            print(indices_replaced.shape)
            print(indices_replaced)
            # probability_matrix[torch.nonzero(indices_replaced == False)[0][0],:]  = 1
            
            # node_attr_token_len , 128
            probability_matrix = torch.zeros((y[i + 1] - y[i],labels.shape[1]))
            print("probability_matrix")
            print(probability_matrix.shape)
            if torch.any(indices_replaced == True):
                print(torch.nonzero(indices_replaced == True))
                probability_matrix[torch.nonzero(indices_replaced == True)[0][0],:]  = 1
            else:
                probability_matrix[rand,:] = 1
            print(probability_matrix)
            prob_matrix.append(probability_matrix)
        prob_matrix = torch.cat(prob_matrix, dim = 0)
        print("prob_matrix")
        print(prob_matrix.shape)
        return prob_matrix

def mask_label(data, masked_node, masked_edge,y,index = None):
        labels = data.clone()
        print("=============")
        print(data.shape)
        # rand = 1
        special_tokens_mask = [get_special_tokens_mask(val).tolist() for val in labels]
        print(len(special_tokens_mask))
        # for i in special_tokens_mask:
        #     print(i.shape)
        if masked_node:
            probability_matrix = mask_node(labels,y, special_tokens_mask)
        if masked_edge:
            probability_matrix = mask_edge(labels,y, special_tokens_mask, index)
        # pdb.set_trace()
        tensor_list = torch.tensor(special_tokens_mask, dtype=torch.int)
        masked_indices = probability_matrix*tensor_list
        masked_indices = masked_indices.bool()
        print("masked_indices")
        print(masked_indices.shape)
        labels[~masked_indices] = -100  # We only compute loss on masked tokens
        # Assert to check if all elements in labels are -100
        assert torch.all(labels == -100), f"Not all elements in labels are -100, {data}, \n {labels}, \n masked_node {masked_node}, masked_edge {masked_edge}"
        return labels, data

In [17]:
import torch
data = torch.load("data.pt")
y = torch.load("y.pt")

mask_label(data, True, False, y)

torch.Size([534, 128])
value
tensor([    0,   302,   301,    67,   348,   264,   377,    16,   262,   352,
          264,   262,   341,   292,   262,   335,   264,  1305,  4762,   555,
         1278, 16489,    16,   262,   309,   264,  2875,   303,     2,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,

AssertionError: Not all elements in labels are -100, tensor([[  0, 302, 301,  ...,   1,   1,   1],
        [  0, 302, 301,  ..., 294,  16,   2],
        [  0, 302, 301,  ...,   1,   1,   1],
        ...,
        [  0, 302, 301,  ...,   1,   1,   1],
        [  0, 302, 301,  ...,   1,   1,   1],
        [  0, 302, 301,  ...,   1,   1,   1]]), 
 tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]]), 
 masked_node True, masked_edge False